In [20]:
# Library Load
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error

# **데이터 가공 및 변환**

In [22]:
path = "data_files/"

bxbooks = pd.read_csv(path+"BX-Books.csv")
bxbookratings = pd.read_csv(path+"BX-Book-Ratings.csv")

print(bxbooks.shape)
print(bxbookratings.shape)

/Users/sunghwanpark/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (3,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(271379, 11)
(999999, 3)


In [23]:
# null 체크
bxbookratings.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [24]:
bxbookratings.head(5)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,155061224,5
2,276727,446520802,0
3,276729,052165615X,3
4,276729,521795028,6


In [25]:
# 데이터 사이즈 조정
bxbookratings=bxbookratings[:50000]
print(bxbookratings.shape)

(50000, 3)


# **도서 간 유사도 산출**

In [26]:
bxbookratings = bxbookratings[['User-ID', 'ISBN', 'Book-Rating']]
bxbookratings_matrix = bxbookratings.pivot_table('Book-Rating', index='User-ID', columns='ISBN')

In [27]:
bxbookratings_matrix

ISBN,0,0 7336 1053 6,0/330/25864/8,00 14 02 5314,000160418X,000221122X,000225669X,000225929X,000617891X,000636988X,...,O451160533,O553375067,O571200338,O590603647,O6712345670,O76790592X,O809463121,"\0210000010""","\0432534220\""","\2842053052\"""
User-ID,,,,,,,,,,,,,,,,,,,,,
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278846,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
278849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
278851,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
# title 컬럼을 얻기 위해 movies와 조인
rating_books = pd.merge(bxbookratings, bxbooks, on='ISBN')

# columns='title'로 title 컬럼으로 피벗 수행.
ratings_matrix = rating_books.pivot_table('Book-Rating', index='User-ID', columns='Book-Title')

# NaN 값을 모두 0으로 변환
ratings_matrix = ratings_matrix.fillna(0)
ratings_matrix.head(3)

Book-Title,It Takes Two,Mystery Mile,Tales of Terror and Suspense,"' Sie belieben wohl zu scherzen, Mr. Feynman.'. Abenteuer eines neugierigen Physikers.",'48,'O Au No Keia: Voices from Hawai'I's Mahu and Transgender Communities,'Salem's Lot,...AND THE HORSE HE RODE IN ON : THE PEOPLE V. KENNETH STARR,".Hack: //Legend of the Twilight, Vol. 1",".Hack: //Legend of the Twilight, Vol. 2",...,eXistenZ: A Novelization,flambeau@darkcorp.com,iI Paradiso Degli Orchi,"metamorfosis, La (ClÃ¡sicos selecciÃ³n series)",stardust,"street bible, the",the Heiress Bride (sister brides),Â¡No bajes al sÃ³tano! (EscalofrÃ­os No. 2),Â¿QuiÃ©n se ha llevado mi queso?,Ã?Â?bermorgen.
User-ID,,,,,,,,,,,,,,,,,,,,,
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
ratings_matrix_T = ratings_matrix.transpose()
ratings_matrix_T.head(3)

User-ID,2,8,9,10,12,14,16,17,19,20,...,278832,278836,278838,278843,278844,278846,278849,278851,278852,278854
Book-Title,,,,,,,,,,,,,,,,,,,,,
It Takes Two,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Mystery Mile,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Tales of Terror and Suspense,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
item_sim = cosine_similarity(ratings_matrix_T, ratings_matrix_T)

# cosine_similarity()로 반환된 Numpy 행렬을 도서명으로 매핑해 DataFrame으로 변환
item_sim_df = pd.DataFrame(data=item_sim, index=ratings_matrix.columns,
                           columns=ratings_matrix.columns)
print(item_sim_df.shape)
item_sim_df.head(3)

(28734, 28734)


Book-Title,It Takes Two,Mystery Mile,Tales of Terror and Suspense,"' Sie belieben wohl zu scherzen, Mr. Feynman.'. Abenteuer eines neugierigen Physikers.",'48,'O Au No Keia: Voices from Hawai'I's Mahu and Transgender Communities,'Salem's Lot,...AND THE HORSE HE RODE IN ON : THE PEOPLE V. KENNETH STARR,".Hack: //Legend of the Twilight, Vol. 1",".Hack: //Legend of the Twilight, Vol. 2",...,eXistenZ: A Novelization,flambeau@darkcorp.com,iI Paradiso Degli Orchi,"metamorfosis, La (ClÃ¡sicos selecciÃ³n series)",stardust,"street bible, the",the Heiress Bride (sister brides),Â¡No bajes al sÃ³tano! (EscalofrÃ­os No. 2),Â¿QuiÃ©n se ha llevado mi queso?,Ã?Â?bermorgen.
Book-Title,,,,,,,,,,,,,,,,,,,,,
It Takes Two,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Mystery Mile,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Tales of Terror and Suspense,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
item_sim_df["Pride and Prejudice"].sort_values(ascending=False)[1:6]

Book-Title
Daddies: All About the Work They Do (Little Golden Books)                    0.707107
Lion in the Valley                                                           0.707107
Tippy-Toe Murder: A Lucy Stone Mystery (Lucy Stone Mysteries (Paperback))    0.707107
The Bungalow Mystery (Nancy Drew Mystery Stories, No 3)                      0.707107
The Truth: A Novel of Discworld                                              0.707107
Name: Pride and Prejudice, dtype: float64

In [32]:
item_sim_df["Romeo and Juliet"].sort_values(ascending=False)[1:6]

Book-Title
She Walks These Hills                         1.0
When I Say No, I Feel Guilty                  1.0
Millie's Book: As Dictated to Barbara Bush    1.0
Mountains of Spices                           1.0
The Girl Who Heard Dragons                    1.0
Name: Romeo and Juliet, dtype: float64

# **개인화된 도서 추천**

In [33]:
# 예측 평점 계산식 함수 구현
def predict_rating(ratings_arr, item_sim_arr):

    # dot : 내적을 이용한 가중합 계산
    ratings_pred = ratings_arr.dot(item_sim_arr) / np.array([np.abs(item_sim_arr).sum(axis=1)])
    return ratings_pred

ratings_pred = predict_rating(ratings_matrix.values, item_sim_df.values)
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index=ratings_matrix.index, 
                                   columns = ratings_matrix.columns)

/Users/sunghwanpark/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """


In [34]:
# 사용자가 평점을 부여한 도서에 대해서만 예측 성능 평가 MSE를 구한다.
def get_mse(pred, actual):
    # 평점이 있는 실제 도서만 추출
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

print('MSE : ', get_mse(ratings_pred, ratings_matrix.values ))

MSE :  15.56994865505014


In [35]:
# 특정 도서와 가장 비슷한 유사도를 가지는 도서에 대해서만 유사도 벡터를 적용하는 예측 평점 계산식 함수 구현
def predict_rating_topsim(ratings_arr, item_sim_arr, n=20):

    # 사용자-아이템 평점 행렬 크기만큼 0으로 채운 예측 행렬 초기화
    pred = np.zeros(ratings_arr.shape)
    
    # 사용자-아이템 평점 행렬의 열 크기만큼 루프 수행.
    for col in range(ratings_arr.shape[1]):
        
        # 유사도 행렬에서 유사도가 큰 순으로 n개 데이터 행렬의 인덱스 반환
        top_n_items = [np.argsort(item_sim_arr[:, col])[:-n-1:-1]]
        
        # 개인화된 예측 평점을 계산
        for row in range(ratings_arr.shape[0]):
            pred[row, col] = item_sim_arr[col, :][top_n_items].dot(ratings_arr[row,
                                                                   :][top_n_items].T)
            pred[row, col] /= np.sum(np.abs(item_sim_arr[col, :][top_n_items]))
            
    return pred

In [36]:
ratings_pred = predict_rating_topsim(ratings_matrix.values, item_sim_df.values, n=20)

print('아이템 기반 최근접 Top-20 이웃 MSE : ', get_mse(ratings_pred, ratings_matrix.values))

# 계산된 예측 평점 데이터를 DataFrame으로 변경
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index=ratings_matrix.index,
                                   columns=ratings_matrix.columns)

/Users/sunghwanpark/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  from ipykernel import kernelapp as app
/Users/sunghwanpark/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  app.launch_new_instance()
/Users/sunghwanpark/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In 

아이템 기반 최근접 Top-20 이웃 MSE :  11.830607624755972


In [37]:
ratings_pred_matrix.head(5)

Book-Title,It Takes Two,Mystery Mile,Tales of Terror and Suspense,"' Sie belieben wohl zu scherzen, Mr. Feynman.'. Abenteuer eines neugierigen Physikers.",'48,'O Au No Keia: Voices from Hawai'I's Mahu and Transgender Communities,'Salem's Lot,...AND THE HORSE HE RODE IN ON : THE PEOPLE V. KENNETH STARR,".Hack: //Legend of the Twilight, Vol. 1",".Hack: //Legend of the Twilight, Vol. 2",...,eXistenZ: A Novelization,flambeau@darkcorp.com,iI Paradiso Degli Orchi,"metamorfosis, La (ClÃ¡sicos selecciÃ³n series)",stardust,"street bible, the",the Heiress Bride (sister brides),Â¡No bajes al sÃ³tano! (EscalofrÃ­os No. 2),Â¿QuiÃ©n se ha llevado mi queso?,Ã?Â?bermorgen.
User-ID,,,,,,,,,,,,,,,,,,,,,
2,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,NaN
8,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,NaN
9,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,NaN
10,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,NaN
12,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,NaN


In [44]:
user_rating_id = ratings_matrix.loc[277928, :]
user_rating_id[ user_rating_id > 0 ].sort_values(ascending=False)[:10]

Book-Title
What the Dogs Have Taught Me, and Other Amazing Things I'Ve Learned          10.0
Mama Makes Up Her Mind: And Other Dangers of Southern Living                 10.0
Depth Takes a Holiday: Essays from Lesser Los Angeles                        10.0
Sleeping at the Starlite Motel: And Other Adventures on the Way Back Home    10.0
Pumping Irony: Working Out the Angst of a Lifetime                           10.0
Italian Neighbors Or, a Lapsed Anglo-Saxon in Verona                         10.0
It Was on Fire When I Lay Down on It                                          9.0
Bill Bryson's African Diary                                                   9.0
Smilla's Sense of Snow                                                        9.0
Lytton Strachey: The New Biography                                            8.0
Name: 277928, dtype: float64

In [52]:
def get_unseen_books(ratings_matrix, userId):
    # userId로 입력받은 사용자의 모든 도서 정보를 추출해 Series로 반환함.
    # 반환된 user_ratings은 도서명(title)을 인덱스로 가지는 Series 객체임.
    user_rating = ratings_matrix.loc[userId, :]
    
    # user_rating이 0보다 크면 기존에 읽은 도서. 대상 인덱스를 추출해 list 객체로 만듦.
    already_seen = user_rating[ user_rating>0 ].index.tolist()
    
    # 모든 도서명을 list 객체로 만듦.
    books_list = ratings_matrix.columns.tolist()
    
    # list comprehension으로 already_seen에 해당하는 도서는 books_list에서 제외함.
    unseen_list = [ book for book in books_list if book not in already_seen ]
    
    return unseen_list

In [54]:
def recomm_movie_by_userid(pred_df, userId, unseen_list, top_n=10):
    # 예측 평점 DataFrame에서 사용자 id 인덱스와 unseen_list로 들어온 도서명 칼럼을 추출해 가장 예측 평점이 높은 순으로 정렬함.
    recomm_books = pred_df.loc[userId, unseen_list].sort_values(ascending=False)[:top_n]
    return recomm_books

# 사용자가 관람하지 않은 도서명 추출
unseen_list = get_unseen_books(ratings_matrix, 277928)

# 아이템 기반의 최근접 이웃 협업 필터링으로 도서 추천
recomm_books = recomm_movie_by_userid(ratings_pred_matrix, 277928, unseen_list, top_n=10)

# 평점 데이터를 DataFrame으로 생성
recomm_books = pd.DataFrame(data=recomm_books.values, index=recomm_books.index, columns=['pred_score'])

recomm_books

,pred_score
Book-Title,
Invasion of Privacy,0.475185
The Butter Did It : A Gastronomic Tale of Love and Murder (Chas Wheatley Mysteries),0.464747
The Witness for the Prosecution and Other Stories,0.464747
Missing in Manhattan: The Adams Round Table,0.464747
Murder at the PTA Luncheon,0.464747
"NIGHT SHE DIED, THE",0.464747
The Deep End of the Ocean (Oprah's Book Club),0.463535
Deep Lie,0.437176
Suspicion of Vengeance,0.311130
